In [31]:
import sqlalchemy
import pandas as pd

from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect, distinct, and_, inspect

import datetime as dt
import requests


In [3]:

engine = create_engine("sqlite:///Earthquake.db")
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)
Base.classes.keys()


['quakes']

In [8]:

conn = engine.connect()
quakes_pd = pd.read_sql("SELECT * FROM quakes", conn)


    

,e_id,date_time,latitude,longitude,depth,mag,magType,nst,gap,dmin,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,0,1975-12-31 14:34:22.080000000,36.813500,-121.132333,5.881,2.71,md,50.0,97.00,0.06757,...,2016-12-15T05:09:40.840Z,Central California,earthquake,0.23,0.640,0.09,15.0,reviewed,nc,nc
1,1,1975-12-31 02:53:56.256000000,34.279000,-116.337000,2.330,2.70,md,0.0,71.00,None,...,2016-01-29T21:39:32.720Z,"16km N of Joshua Tree, CA",earthquake,None,0.847,None,0.0,reviewed,ci,ci
2,2,1975-12-30 23:37:52.896000000,36.905667,-121.637167,-0.155,2.67,md,13.0,137.00,0.03243,...,2016-12-15T05:08:54.636Z,Central California,quarry blast,0.34,1.060,0.77,3.0,reviewed,nc,nc
3,3,1975-12-30 23:04:47.424000000,36.838667,-121.569833,4.774,2.67,md,46.0,81.00,0.01892,...,2016-12-15T05:08:50.998Z,Central California,earthquake,0.21,0.360,0.28,11.0,reviewed,nc,nc
4,4,1975-12-30 18:10:28.128000000,35.781000,-121.421000,11.895,3.15,md,17.0,247.00,0.2396,...,2016-12-15T05:08:39.866Z,offshore Central California,earthquake,1.04,0.610,0.55,4.0,reviewed,nc,nc
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80382,80382,2016-01-03 09:12:46.368000000,41.848300,-119.601600,6.700,2.50,ml,8.0,75.42,0.179,...,2018-07-02T17:44:50.027Z,"72km ESE of Lakeview, Oregon",earthquake,None,7.200,0.34,5.0,reviewed,nn,nn
80383,80383,2016-01-03 04:34:59.808000000,41.869000,-119.613700,9.900,2.50,ml,8.0,99.14,0.159,...,2018-07-02T17:44:48.976Z,"70km ESE of Lakeview, Oregon",earthquake,None,4.200,0.21,5.0,reviewed,nn,nn
80384,80384,2016-01-02 06:38:32.928000000,38.085667,-121.916000,20.430,2.50,md,68.0,83.00,0.0982,...,2017-02-09T18:12:19.290Z,"6km NNW of Pittsburg, California",earthquake,0.26,0.190,0.155,71.0,reviewed,nc,nc
80385,80385,2016-01-01 20:02:39.552000000,40.175500,-122.228833,32.750,2.56,md,20.0,124.00,0.08559,...,2017-02-09T18:04:27.831Z,"0km ESE of Red Bluff, California",earthquake,0.83,1.370,0.154,47.0,reviewed,nc,nc


In [16]:
lon = -120
lat = 36
radius = 30
start_date = '2016-01-01'
end_date = '2017-05-05'

cols = 'date_time, latitude, longitude, depth, mag'
sql = f"SELECT * FROM quakes where {start_date} < date_time and date_time < {end_date}"
quakes_pd = pd.read_sql(sql, conn)

quakes_pd


,e_id,date_time,latitude,longitude,depth,mag,magType,nst,gap,dmin,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource


## Retrieve last 30 days function

In [48]:


def ReadUSGS(BaseDate = dt.datetime.today()):
    
    StartDate = (BaseDate - dt.timedelta(days=30)).strftime('%Y-%m-%d %H:%M:%S')
    EndDate = BaseDate.strftime('%Y-%m-%d %H:%M:%S')
    
    url = f'https://earthquake.usgs.gov/fdsnws/event/1/query.geojson?starttime={StartDate}&endtime={EndDate}&maxlatitude=41.961&minlatitude=32.813&maxlongitude=-114.521&minlongitude=-124.255&minmagnitude=2.5&orderby=time'
    
    data = requests.get(url)
    return data.json()

In [50]:
ReadUSGS('2020-10-19')

TypeError: unsupported operand type(s) for -: 'str' and 'datetime.timedelta'

In [19]:
(dt.datetime.today() - dt.timedelta(days=30)).strftime('%Y-%m-%d')


'2020-10-19'

In [54]:

a = dt.datetime.strptime('2020-10-19 00:00:00', '%Y-%m-%d %H:%M:%S')
a

datetime.datetime(2020, 10, 19, 0, 0)

In [58]:
def predict(AtDate):

    myDate = dt.datetime.strptime(AtDate, '%Y-%m-%d %H:%M:%S')

    return ReadUSGS(myDate)

In [59]:
predict('2020-10-19 00:00:00')

{'type': 'FeatureCollection', 'metadata': {'generated': 1605754639000, 'url': 'https://earthquake.usgs.gov/fdsnws/event/1/query.geojson?starttime=2020-09-19%2000:00:00&endtime=2020-10-19%2000:00:00&maxlatitude=41.961&minlatitude=32.813&maxlongitude=-114.521&minlongitude=-124.255&minmagnitude=2.5&orderby=time', 'title': 'USGS Earthquakes', 'status': 200, 'api': '1.10.3', 'count': 261}, 'features': [{'type': 'Feature', 'properties': {'mag': 3.25, 'place': '7km ENE of Petrolia, CA', 'time': 1603004517140, 'updated': 1605338848040, 'tz': None, 'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/nc73472226', 'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=nc73472226&format=geojson', 'felt': 7, 'cdi': 2.2, 'mmi': None, 'alert': None, 'status': 'reviewed', 'tsunami': 0, 'sig': 164, 'net': 'nc', 'code': '73472226', 'ids': ',nc73472226,us6000c94y,', 'sources': ',nc,us,', 'types': ',dyfi,focal-mechanism,nearby-cities,origin,phase-data,scitech-link,', 'nst': 29, 'dmin': 